# Addictive Model

## Libraries

In [ ]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from pulp import LpProblem, LpVariable, LpMinimize, lpSum, LpStatus

## Import Dataset

In [ ]:
file_path = '../data/food_data_cleaned.xlsx'
food_data = pd.ExcelFile(file_path)

In [ ]:
data = food_data.parse('Sheet1')
data.head()

,Food Name,Energy (KJ/100g),Sugars (g/100g),Saturated Fat (g/100g),Salt (g/100g),Proteins (g/100g),Fiber (g/100g),Fruit/Veg (%),Nutri-Score,Ecoscore
0,Nutella,2252.0,56.3,10.6,0.107,6.3,0.0,0.0,e,21
1,Sésame,1961.0,17.0,2.0,0.380,10.0,4.6,0.0,b,57
2,Almonds,2567.0,4.8,4.3,0.010,24.5,12.1,100.0,a,24
3,Alvalle Gazpacho l'original,168.0,3.3,0.4,0.620,0.9,1.2,0.0,a,82
4,70% Cacao noir intense,2350.0,30.0,24.0,0.100,9.5,0.0,0.0,e,29


Preprocessing

In [ ]:
nutri_score_mapping = {'a': 5, 'b': 4, 'c': 3, 'd': 2, 'e': 1}
data['Nutri-Score'] = data['Nutri-Score'].map(nutri_score_mapping)

In [ ]:
data

,Food Name,Energy (KJ/100g),Sugars (g/100g),Saturated Fat (g/100g),Salt (g/100g),Proteins (g/100g),Fiber (g/100g),Fruit/Veg (%),Nutri-Score,Ecoscore
0,Nutella,2252.0,56.3,10.6,0.107,6.3,0.0,0.0,1,21
1,Sésame,1961.0,17.0,2.0,0.380,10.0,4.6,0.0,4,57
2,Almonds,2567.0,4.8,4.3,0.010,24.5,12.1,100.0,5,24
3,Alvalle Gazpacho l'original,168.0,3.3,0.4,0.620,0.9,1.2,0.0,5,82
4,70% Cacao noir intense,2350.0,30.0,24.0,0.100,9.5,0.0,0.0,1,29
...,...,...,...,...,...,...,...,...,...,...
715,Tomato ketchup,435.0,22.8,0.1,1.800,1.2,0.0,0.0,2,85
716,Cola Cao El Original,288.0,9.1,1.1,0.120,3.4,0.5,0.0,4,0
717,Gnocchi à poêler,801.0,1.5,0.3,1.100,5.0,2.6,0.0,4,82
718,Tartines craquantes à la châtaigne,1616.0,6.5,0.5,0.300,6.7,4.3,0.0,5,75


## Weighted Model

The weighted sum model combines multiple criteria into a single score. Here, we combine **Nutri-Score** (nutritional value) and **Eco-Score** (environmental impact).

Formula:
\[ F(x) = w_1 \times \text{Nutri-Score} + w_2 \times \text{Eco-Score} \]

Where:
- \( F(x) \): Comprehensive score.
- \( w_1, w_2 \): Weights for Nutri-Score and Eco-Score.
- Nutri-Score: Nutritional value.
- Eco-Score: Environmental impact.

Weight Selection:
- \( w_1 > w_2 \) if health is prioritized.
- \( w_1 = w_2 \) for equal importance.

Advantages:
- **Simple**: Easy to compute.
- **Flexible**: Adjust weights for priorities.
- **Transparent**: Clear score combination.

Limitations:
- **Linearity**: Cannot capture complex interactions.
- **Weight Sensitivity**: Scores depend on weights.

In [ ]:
w1, w2 = 0.6, 0.4

# Calculate weighted sum score
data['Comprehensive Score'] = w1 * data['Nutri-Score'] + w2 * data['Ecoscore']

min_score = data['Comprehensive Score'].min()
max_score = data['Comprehensive Score'].max()
data['Scaled Score'] = 1 + 4 * (data['Comprehensive Score'] - min_score) / (max_score - min_score)

data_sorted = data.sort_values(by='Scaled Score', ascending=False)

print(data_sorted[['Food Name', 'Nutri-Score', 'Ecoscore', 'Scaled Score']])
data_sorted.to_csv('weighted_sum_scaled_results.csv', index=False)

                                            Food Name  Nutri-Score  Ecoscore  \
608                           BISCOTTES COMPLETES BIO            5       110   
10                                      Pulco Zitrone            3       112   
576                                     Le Bio Orange            2       111   
476  Pain de seigle complet Epeautre et épeautre vert            5       102   
258                                       Tofoo Naked            5       101   
..                                                ...          ...       ...   
39                            1848 Chocolat en poudre            5       -16   
166                          Grand Arôme 32% de Cacao            2       -15   
566                                chocolat en poudre            2       -17   
193                                          Le Choco            4       -20   
150                                            ricore            1       -20   

     Scaled Score  
608      5.000000  

## Rule Based Model

Rule-Based Model

Definition of Classification Rules:

The classification model divides food items into categories based on their Nutri-Score (A to E) and Eco-Score (0 to 100). The updated rules for categorization are:

- **Premium**: Food items where Nutri-Score is "A" and Eco-Score falls into the highest category (e.g., A or above 80).
- **Good**: Food items where either Nutri-Score is "A" or Eco-Score is above 60, but not both.
- **Average**: Food items where Nutri-Score is "B" and Eco-Score falls between 40 and 60.
- **Poor**: Food items where Nutri-Score is "C" or "D" and Eco-Score is below 40.
- **Low**: Food items where both Nutri-Score is "E" and Eco-Score is below 20.

Conflict Resolution:
To resolve conflicts when Nutri-Score and Eco-Score indicate different categories:
1. **Health First Rule**: When the focus is health, prioritize Nutri-Score over Eco-Score.
2. **Environment First Rule**: When the focus is environmental impact, prioritize Eco-Score over Nutri-Score.
3. **Default Conflict Rules**:
   - When Nutri-Score is "A" and Eco-Score is below 40, classify as "Good."
   - When Eco-Score is above 80 and Nutri-Score is "D" or "E," classify as "Average."

Advantages of Updated Rules:
1. **Granularity**: More nuanced distinctions between food items.
2. **Conflict Resolution**: Clear and logical rules to address conflicting scores.
3. **Flexibility**: Easy adjustment for varying priorities (health vs. environment).

Limitations:
1. **Complexity**: Slightly more complex than simple classification rules.
2. **Weight Sensitivity**: Results may vary with Eco-Score thresholds and Nutri-Score categories.

In [ ]:
def classify_food(row):
    nutri_score = row["Nutri-Score"]
    eco_score = row["Ecoscore"]
    
    # Classification logic
    if nutri_score == "A" and eco_score > 80:
        return "Premium"
    elif (nutri_score == "A" or eco_score > 60) and not (nutri_score == "A" and eco_score > 80):
        return "Good"
    elif nutri_score == "B" and 40 <= eco_score <= 60:
        return "Average"
    elif nutri_score in ["C", "D"] and eco_score < 40:
        return "Poor"
    elif nutri_score == "E" and eco_score < 20:
        return "Low"
    else:
        return "Average"

data["Category"] = data.apply(classify_food, axis=1)

print(data[["Food Name", "Nutri-Score", "Ecoscore", "Category"]])

                              Food Name  Nutri-Score  Ecoscore Category
0                               Nutella            1        21  Average
1                                Sésame            4        57  Average
2                               Almonds            5        24  Average
3           Alvalle Gazpacho l'original            5        82     Good
4                70% Cacao noir intense            1        29  Average
..                                  ...          ...       ...      ...
715                      Tomato ketchup            2        85     Good
716                Cola Cao El Original            4         0  Average
717                    Gnocchi à poêler            4        82     Good
718  Tartines craquantes à la châtaigne            5        75     Good
719                           Haselnuss            1        58  Average

[720 rows x 4 columns]
